In [9]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline

## CRO100 Problem definition

In [3]:
CRO100A = "1380,939,2848,96,3510,1671,457,334,3888,666,984,965,2721,1482,1286,525,2716,1432,738,1325,1251,1832,2728,1698,3815,169,3683,1533,1247,1945,123,862,1234,1946,252,1240,611,673,2576,1676,928,1700,53,857,1807,1711,274,1420,2574,946,178,24,2678,1825,1795,962,3384,1498,3520,1079,1256,61,1424,1728,3913,192,3085,1528,2573,1969,463,1670,3875,598,298,1513,3479,821,2542,236,3955,1743,1323,280,3447,1830,2936,337,1621,1830,3373,1646,1393,1368,3874,1318,938,955,3022,474,2482,1183,3854,923,376,825,2519,135,2945,1622,953,268,2628,1479,2097,981,890,1846,2139,1806,2421,1007,2290,1810,1115,1052,2588,302,327,265,241,341,1917,687,2991,792,2573,599,19,674,3911,1673,872,1559,2863,558,929,1766,839,620,3893,102,2178,1619,3822,899,378,1048,1178,100,2599,901,3416,143,2961,1605,611,1384,3113,885,2597,1830,2586,1286,161,906,1429,134,742,1025,1625,1651,1187,706,1787,1009,22,987,3640,43,3756,882,776,392,1724,1642,198,1810,3950,1558"
A = np.fromstring(CRO100A, sep=',').reshape((100, 2))
A = np.sqrt(np.subtract.outer(A[:,0], A[:,0])**2 + np.subtract.outer(A[:,1], A[:,1])**2)

CRO100B = "3140,1401,556,1056,3675,1522,1182,1853,3595,111,962,1895,2030,1186,3507,1851,2642,1269,3438,901,3858,1472,2937,1568,376,1018,839,1355,706,1925,749,920,298,615,694,552,387,190,2801,695,3133,1143,1517,266,1538,224,844,520,2639,1239,3123,217,2489,1520,3834,1827,3417,1808,2938,543,71,1323,3245,1828,731,1741,2312,1270,2426,1851,380,478,2310,635,2830,775,3829,513,3684,445,171,514,627,1261,1490,1123,61,81,422,542,2698,1221,2372,127,177,1390,3084,748,1213,910,3,1817,1782,995,3896,742,1829,812,1286,550,3017,108,2132,1432,2000,1110,3317,1966,1729,1498,2408,1747,3292,152,193,1210,782,1462,2503,352,1697,1924,3821,147,3370,791,3162,367,3938,516,2741,1583,2330,741,3918,1088,1794,1589,2929,485,3453,1998,896,705,399,850,2614,195,2800,653,2630,20,563,1513,1090,1652,2009,1163,3876,1165,3084,774,1526,1612,1612,328,1423,1322,3058,1276,3782,1865,347,252,3904,1444,2191,1579,3220,1454,468,319,3611,1968,3114,1629,3515,1892,3060,155"
B = np.fromstring(CRO100B, sep=',').reshape((100, 2))
B = np.sqrt(np.subtract.outer(B[:,0], B[:,0])**2 + np.subtract.outer(B[:,1], B[:,1])**2)

## Objective function

In [16]:
def tsp_objective_function(p, M):
    s = 0.0
    for i in range(100):
        s += M[p[i-1], p[i]]
    return s

def tsp_objective_matrix_A(xs):
    return np.apply_along_axis(tsp_objective_function, 1, xs, A)

def tsp_objective_matrix_B(xs):
    return np.apply_along_axis(tsp_objective_function, 1, xs, B)

## Sequence operators

In [6]:
def PMX(ind1, ind2):

    size = min(len(ind1), len(ind2))
    p1, p2 = np.zeros(size, dtype='int'), np.zeros(size, dtype='int')

    # Initialize the position of each indices in the individuals
    for i in xrange(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = np.random.randint(0, size)
    cxpoint2 = np.random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1
    
    # Apply crossover between cx points
    for i in xrange(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    return ind1, ind2

def reverse_sequence_mutation(p):
    a = np.random.choice(len(p), 2, False)
    i, j = a.min(), a.max()
    q = p.copy()
    q[i:j+1] = q[i:j+1][::-1]
    return q

## SGA-II alogrithm

In [5]:
def front_pareto(population, evaluations):
    nup = population.shape[0]
    domination_count = np.zeros(nup)
    pareto_front = np.empty(nup)
    dominatees = []
    f = []
    for i in xrange(nup):
        dominatees.append(np.nonzero(np.all(evaluations > evaluations[i], axis=1))[0])
        domination_count[i] = np.sum(np.all(evaluations < evaluations[i], axis=1))
        if domination_count[i] == 0:
            pareto_front[i] = 1
            f.append(i)
    
    fr = 2
    while f:
        nxt = []
        for p in f:
            for d in dominatees[p]:
                domination_count[d] -= 1
                if domination_count[d] == 0:
                    pareto_front[d] = fr
                    nxt.append(d)
        fr += 1
        f = nxt
    return pareto_front

def NSGA2_sort(population, evaluations):
    nup = population.shape[0]
    nf = evaluations.shape[1]
    pareto_front = front_pareto(population, evaluations)
    distances = np.zeros(nup)
        
    for i in xrange(nf):
        idxs = np.argsort(evaluations[:, i])
        population = population[idxs]
        evaluations = evaluations[idxs]
        pareto_front = pareto_front[idxs]
        distances = distances[idxs]
        distances[0] = np.inf
        distances[nup-1] = np.inf
        distances[1:nup-1] += evaluations[2:, i] - evaluations[:-2, i]

    idxs = np.lexsort((-distances, pareto_front))
    return population[idxs], evaluations[idxs]

def tournament_selection(population_size, how_many, t_size = 5):
    return np.random.choice(population_size, how_many * t_size).reshape((how_many, t_size)).min(axis=1)

In [21]:
def NSGA2(iterations, genome_length, population_size, offspring_size, evaluation_functions, mutation_chance):
    
    log_populations = np.empty((iterations, population_size, genome_length), dtype=np.int32)
    
    current_population_solutions = np.empty((population_size, genome_length), dtype=np.int32)
    for i in xrange(population_size):
        current_population_solutions[i] = np.random.permutation(genome_length)
        
    current_population_scores = np.empty((population_size, len(evaluation_functions)))
    
    for f in xrange(len(evaluation_functions)):
        current_population_scores[:, f] = evaluation_functions[f](current_population_solutions).T
        
    current_population_solutions, current_population_scores = NSGA2_sort(current_population_solutions, current_population_scores)
    
    for i in tqdm_notebook(xrange(iterations)):
        # parent selection
        par1Idx = tournament_selection(population_size, offspring_size)
        par2Idx = tournament_selection(population_size, offspring_size)
        
        # crossover
        offspring_population_solutions = np.empty((offspring_size, genome_length), dtype=np.int32)
        for off in xrange(offspring_size):
            offspring_population_solutions[off] = PMX(current_population_solutions[par1Idx[off]], current_population_solutions[par2Idx[off]])[0]
        
        # mutation
        mutation_indicator = np.random.rand(offspring_size, genome_length) < mutation_chance
        offspring_population_solutions[mutation_indicator] = np.apply_along_axis(reverse_sequence_mutation, 1, offspring_population_solutions)[mutation_indicator]
        
        # evaluation of offspring
        offspring_population_scores = np.empty((offspring_size, len(evaluation_functions)))
        for f in xrange(len(evaluation_functions)):
            offspring_population_scores[:, f] = evaluation_functions[f](offspring_population_solutions).T
        
        combined_solutions, combined_evaluations = NSGA2_sort(np.vstack([current_population_solutions, offspring_population_solutions]), np.vstack([current_population_scores, offspring_population_scores]))
        current_population_solutions = combined_solutions[:population_size]
        current_population_scores = combined_evaluations[:population_size]
        log_populations[i, :, :] = current_population_solutions[:, :]
        
    return log_populations
        

In [8]:
def graph_population(population, evaluation_functions):
    scores = np.empty((population.shape[0], len(evaluation_functions)))
    for f in xrange(len(evaluation_functions)):
        scores[:, f] = evaluation_functions[f](population).T
    
    par = front_pareto(population, scores)
    lowest = scores[par == 1]
    
    plt.plot(figsize=(18, 6))
    plt.scatter(scores[:, 0], scores[:, 1], c=par)
    plt.plot(lowest[:, 0], lowest[:, 1], 'ro')
    plt.xlabel('1st fitness function')
    plt.ylabel('2nd fitness function')
    plt.show()

In [22]:
iteration_count = 100
genome_length = 100
population_size = 300
offspring_count = 900
ev_fun_cro100 = np.array([tsp_objective_matrix_A, tsp_objective_matrix_B])
mutation_chance = 0.1
pops_cro100 = NSGA2(iteration_count, genome_length, population_size, offspring_count, ev_fun_cro100, mutation_chance)

HBox(children=(IntProgress(value=0), HTML(value=u'')))

In [23]:
def f_cro100(x):
    graph_population(pops_cro100[x], ev_fun_cro100)
    
x = interact(f_cro100, x=widgets.IntSlider(min=0,max=99,step=1,value=0))

interactive(children=(IntSlider(value=0, description=u'x', max=99), Output()), _dom_classes=('widget-interact',))